## CmdStanPyの動作確認

### CmdStanPy公式の"Hello World"

In [1]:
# import packages
import os
from cmdstanpy import cmdstan_path, CmdStanModel

# specify Stan program file
bernoulli_stan = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.stan')

# instantiate the model; compiles the Stan program as needed.
bernoulli_model = CmdStanModel(stan_file=bernoulli_stan)

# inspect model object
print(bernoulli_model)

CmdStanModel: name=bernoulli
	 stan_file=/root/.cmdstan/cmdstan-2.30.0/examples/bernoulli/bernoulli.stan
	 exe_file=/root/.cmdstan/cmdstan-2.30.0/examples/bernoulli/bernoulli
	 compiler_options=stanc_options={}, cpp_options={}


In [2]:
# specify data file
bernoulli_data = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.data.json')

# fit the model
bern_fit = bernoulli_model.sample(data=bernoulli_data, output_dir='.')

# printing the object reports sampler commands, output files
print(bern_fit)

20:28:34 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

20:28:35 - cmdstanpy - INFO - CmdStan done processing.



CmdStanMCMC: model=bernoulli chains=4['method=sample', 'algorithm=hmc', 'adapt', 'engaged=1']
 csv_files:
	/workdir/demo/bernoulli-20220709202834_1.csv
	/workdir/demo/bernoulli-20220709202834_2.csv
	/workdir/demo/bernoulli-20220709202834_3.csv
	/workdir/demo/bernoulli-20220709202834_4.csv
 output_files:
	/workdir/demo/bernoulli-20220709202834_0-stdout.txt
	/workdir/demo/bernoulli-20220709202834_1-stdout.txt
	/workdir/demo/bernoulli-20220709202834_2-stdout.txt
	/workdir/demo/bernoulli-20220709202834_3-stdout.txt


In [3]:
bern_fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-7.261260,0.019761,0.706144,-8.701150,-6.989320,-6.749720,1276.99,510.386,1.00175
theta,0.245416,0.003084,0.117830,0.079008,0.232547,0.462923,1459.87,583.480,1.00243


In [4]:
print(bern_fit.diagnose())

Processing csv files: /workdir/demo/bernoulli-20220709202834_1.csv, /workdir/demo/bernoulli-20220709202834_2.csv, /workdir/demo/bernoulli-20220709202834_3.csv, /workdir/demo/bernoulli-20220709202834_4.csv

Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
No divergent transitions found.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory.

Effective sample size satisfactory.

Split R-hat values satisfactory all parameters.

Processing complete, no problems detected.



### 8schools
- pystanとの速度比較

In [5]:
%%time
model = CmdStanModel(stan_file="8schools.stan")

20:28:49 - cmdstanpy - INFO - compiling stan file /workdir/demo/8schools.stan to exe file /workdir/demo/8schools
20:29:03 - cmdstanpy - INFO - compiled model executable: /workdir/demo/8schools


CPU times: user 6.57 ms, sys: 1.86 ms, total: 8.43 ms
Wall time: 14.3 s


- 15s程度でコンパイルできた。
- pystan2では1min超を要したから、確かにコンパイルはかなり高速と言える。
  - コンパイルしたモデルは保存されており、2回目以降は数秒で読み込める。(`8schools`と`8schools.hpp`を削除すれば初回同様となる)

In [6]:
%%time
data = "8schools.data.json"
fit = model.sample(data=data, iter_sampling=1000, iter_warmup=500)

20:29:05 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

20:29:06 - cmdstanpy - INFO - CmdStan done processing.
20:29:06 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 1 had 5 divergent transitions (0.5%)
	Use function "diagnose()" to see further information.



CPU times: user 190 ms, sys: 26.7 ms, total: 217 ms
Wall time: 427 ms


- 200ms強を要しており、微差ではあるがPyStan2（163ms）よりやや遅い？

In [7]:
fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-4.923760,0.072446,2.604700,-9.697300,-4.669300,-1.11580,1292.670,5477.43,1.001900
mu,8.111190,0.188841,5.399530,-0.175280,7.945290,17.01660,817.559,3464.23,1.000230
tau,6.566660,0.162449,5.385040,0.646541,5.283470,17.13370,1098.860,4656.19,1.003110
eta[1],0.404048,0.015845,0.933542,-1.161070,0.407255,1.92579,3471.060,14707.90,1.000520
eta[2],-0.025077,0.015460,0.922999,-1.536050,-0.031319,1.51418,3564.280,15102.90,1.000190
eta[3],-0.198637,0.014264,0.917975,-1.711680,-0.212369,1.35024,4141.770,17549.90,0.999530
eta[4],-0.058792,0.015147,0.905125,-1.534910,-0.082534,1.43891,3571.040,15131.50,0.999681
eta[5],-0.354350,0.017371,0.881159,-1.734490,-0.372865,1.11117,2573.250,10903.60,1.000020
eta[6],-0.224962,0.014916,0.900132,-1.713310,-0.255550,1.31839,3641.570,15430.40,0.999872


In [8]:
print(fit.diagnose())

Processing csv files: /tmp/tmpdgu8jduq/8schoolsyvoajs_d/8schools-20220709202906_1.csv, /tmp/tmpdgu8jduq/8schoolsyvoajs_d/8schools-20220709202906_2.csv, /tmp/tmpdgu8jduq/8schoolsyvoajs_d/8schools-20220709202906_3.csv, /tmp/tmpdgu8jduq/8schoolsyvoajs_d/8schools-20220709202906_4.csv

Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
5 of 4000 (0.12%) transitions ended with a divergence.
These divergent transitions indicate that HMC is not fully able to explore the posterior distribution.
Try increasing adapt delta closer to 1.
If this doesn't remove all divergences, try to reparameterize the model.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory.

Effective sample size satisfactory.

Split R-hat values satisfactory all parameters.

Processing complete.

